# AES y Diffie-Hellman para solucionar la transmisión de datos entre un sensor de temperatura y la nube de ACME.

## Integrantes
- Mauricio Carvajal Andrade      |     mauricio.carvajala@sansano.usm.cl    |
- Diego Prokes Herbage           |     diego.prokes@usm.cl                  |
- Alex Aravena Tapia             |     alex.aravenata@usm.cl                |
- Benjamín Mena Ardura           |     benjamin.menaa@usm.cl                |

## Bibliotecas utilizadas

Se utilizó la biblioteca de Cryptodome, su instalación se realiza mediante el comando
```
pip install pycryptodome==3.19.0
```

## Caso sin seguridad implementada
En este caso se tiene la transmisión de datos desde un sensor hacia el servidor

In [9]:
# Caso sin seguridad

class CanalComunicacion:
    def __init__(self, datos=None):
        self.datos = datos
    # def __init__(self, clave_compartida):
    #     self.clave_compartida = clave_compartida

    def recibir_datos(self, datos):
        self.datos = datos
        return 0

    # El canal transmite los datos
    def enviar_datos(self):
        return self.datos

    # def enviar_datos_con_seguridad(self, datos):
    #     # Aquí implementarías la lógica de cifrado con la clave compartida
    #     return datos  # Simulación: devolvemos los datos sin cifrar

    # def recibir_datos_con_seguridad(self, datos_cifrados):
    #     # Aquí implementarías la lógica de descifrado con la clave compartida
    #     return datos_cifrados  # Simulación: devolvemos los datos sin descifrar

    def print_datos(self):
        print(self.datos)
        return 0
    
class Sensor:
    def __init__(self, datos):
        self.datos = datos

    # El sensor envía datos en texto plano por el canal
    def enviar_datos_sin_seguridad(self, canal):
        datos_cifrados = canal.recibir_datos(self.datos)
        return datos_cifrados
    
    def print_datos(self):
        print(self.datos)
        return 0
    
    def return_datos(self):
        return self.datos
    
class Servidor:
    def __init__(self, datos_recibidos=None):
        self.datos_recibidos = datos_recibidos

    # El sensor recibe datos que vienen en texto plano 
    def recibir_datos_sin_seguridad(self, canal):
        self.datos_recibidos = canal.enviar_datos()
        return 0

    def print_datos_recibidos(self):
        print(self.datos_recibidos)
        return 0
    
    def return_datos_recibidos(self):
        return self.datos_recibidos
    
class ManInTheMiddle:
    def __init__(self, datos_interceptados=None):
        self.datos_interceptados = datos_interceptados

    # El MITM recibe los datos que interceptó del canal
    def recibir_datos(self, canal):
        self.datos_interceptados = canal.enviar_datos()
        return 0
    
    # El MITM modifica los datos que interceptó del canal
    def modificar_datos_interceptados(self, datos_modificados):
        self.datos_interceptados = datos_modificados
        return 0

    # El MITM envía datos nuevos al canal
    def enviar_datos_sin_seguridad(self, canal):
        canal.recibir_datos(self.datos_interceptados)
        return 0

    def print_datos_interceptados(self):
        print(self.datos_interceptados)
        return 0
    
    def return_datos_interceptados(self):
        return self.datos_interceptados

datos_del_sensor = 34 # temp. en grados Celsius

# instancias
canal = CanalComunicacion()
sensor = Sensor(datos_del_sensor)
servidor = Servidor()
mitm = ManInTheMiddle()

# se verifican los datos que tiene el sensor
print("El sensor envía que la tempreatura que midió es: " + str(sensor.return_datos()) + " grados.")

# el emisor envía los datos al canal
sensor.enviar_datos_sin_seguridad(canal)

# el man in the middle intercepta los datos del canal
mitm.recibir_datos(canal)

# se modifican los datos que interceptó man in the middle
mitm.modificar_datos_interceptados(38)

# se modifican los datos que interceptó man in the middle
mitm.enviar_datos_sin_seguridad(canal)

# el servidor recibe los datos del canal
servidor.recibir_datos_sin_seguridad(canal)

# el servidor imprime los datos que recibió
print("El servidor recibió que la temperatura medida por el sensor es: " + str(servidor.return_datos_recibidos()) + " grados.")

El sensor envía que la tempreatura que midió es: 34 grados.
El servidor recibió que la temperatura medida por el sensor es: 38 grados.


## Caso con seguridad implementada

In [8]:
from Crypto.Cipher import AES
from Crypto.PublicKey import ECC
from Crypto.Hash import SHAKE128
from Crypto.Protocol.DH import key_agreement
import os

class Sensor:
    def __init__(self, datos):
        self.datos = datos

    def enviar_datos_con_seguridad(self, canal, session_key):
        nonce = os.urandom(13)  # Genera un nonce aleatorio de 13 bytes
        cipher = AES.new(session_key, AES.MODE_CCM, nonce=nonce)
        ciphertext, tag = cipher.encrypt_and_digest(self.datos)
        
        # Simulación: enviar datos cifrados, tag y nonce al canal
        canal.recibir_datos(ciphertext, tag, nonce)

class Servidor:
    def __init__(self):
        self.datos_recibidos = None

    def recibir_datos_con_seguridad(self, canal, session_key):
        try:
            # Simulación: recibir datos cifrados, tag y nonce del canal
            ciphertext, tag, nonce = canal.enviar_datos()
            
            cipher = AES.new(session_key, AES.MODE_CCM, nonce=nonce)
            datos_decifrados = cipher.decrypt_and_verify(ciphertext, tag)

            self.datos_recibidos = datos_decifrados

        except ValueError:
            print("¡Alerta! El mensaje ha sido alterado.")

class Atacante:
    def modificar_datos(self, canal):
        # Simulación: el atacante intenta modificar los datos
        ataque_exitoso = False
        if canal.datos is not None:
            canal.datos = b"Dato_modificado_por_atacante"
            ataque_exitoso = True

        return ataque_exitoso

# Generar una clave de sesión usando Diffie-Hellman
def generar_clave_session():
    U = ECC.generate(curve='p256')
    V = ECC.generate(curve='p256').public_key()
    session_key = key_agreement(static_priv=U, static_pub=V, kdf=kdf)
    return session_key

# Esta KDF ha sido acordada previamente
def kdf(x):
    return SHAKE128.new(x).read(16)

class CanalComunicacion:
    def __init__(self):
        self.datos = None
        self.tag = None
        self.nonce = None

    def recibir_datos(self, datos, tag, nonce):
        self.datos = datos
        self.tag = tag
        self.nonce = nonce

    def enviar_datos(self):
        return self.datos, self.tag, self.nonce

# Datos a enviar desde el sensor al servidor
datos_del_sensor = b"Hola, esto es un mensaje desde el sensor."

# Generar la clave de sesión
session_key = generar_clave_session()

# Instancias
canal = CanalComunicacion()
sensor = Sensor(datos_del_sensor)
servidor = Servidor()
atacante = Atacante()

# El sensor envía datos cifrados con seguridad al canal
sensor.enviar_datos_con_seguridad(canal, session_key)

# El atacante intenta modificar los datos
ataque_exitoso = atacante.modificar_datos(canal)
if ataque_exitoso:
    print("Ataque exitoso: Datos modificados por el atacante.")

# El servidor recibe los datos cifrados con seguridad del canal
servidor.recibir_datos_con_seguridad(canal, session_key)

# El servidor imprime los datos recibidos
print("Datos recibidos por el servidor:", servidor.datos_recibidos)

Datos recibidos por el servidor: b'Hola, esto es un mensaje desde el sensor.'
